## 03f - Output: Regression analysis with linked survey data

- **Project:** _Families, households, networks: Rethinking the relational structure of families through large-scale network data_ <br>
- **Authors:** Nicolás Soler (ORCID 0009-0001-4239-9396), Tom Emery, Agnieszka Kanas <br>
- **Last updated:** January 2026 <br>
- **Full research article published in journal:** _Demography_ (2026)

In [ ]:
import yaml
import polars as pl
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

In [ ]:
# Load YAML configuration
path_config = "config.yml"
with open(path_config, "r") as f:
    config = yaml.safe_load(f)

In [ ]:
# Read sample
dtypes_sample = {
    "RINPERSOON":pl.String,
    "nomem_encr":pl.String,
    "id_hhd":pl.Int64,
    "is_ego_survey":pl.Int64,
    "hhd_size":pl.Int64,
    "net_size_hhd_1":pl.Int64,
    "net_size_hhd_2":pl.Int64,
    "net_size_hhd_3":pl.Int64,
    "net_size_hhd_4":pl.Int64,
    "density_2":pl.Float64,
    "density_3":pl.Float64,
    "density_4":pl.Float64,
    "overlap_survey_1":pl.Int64,
    "overlap_survey_2":pl.Int64,
    "overlap_survey_3":pl.Int64,
    "overlap_survey_4":pl.Int64,
}

sample = pl.scan_csv(config["data"]["sample"], separator=",", encoding="utf8", schema_overrides=dtypes_sample).select(dtypes_sample.keys()).collect()

In [ ]:
# Read outcomes

dtypes_outcomes = {
    "nomem_encr":pl.String,
    "y_perc_fam_comp":pl.Float64,
    "y_perc_contact_comp":pl.Float64,
    "y_perc_supp_comp":pl.Float64
}

outcomes = (
    pl
    .scan_csv(config["data"]["survey_outcomes"], separator=",", encoding="utf8", schema_overrides=dtypes_outcomes)
    .select(dtypes_outcomes.keys())
    .collect()
)

# 1 - Data preparation

In [ ]:
# Merge outcomes to sample
data = sample.join(outcomes, how="inner", on="nomem_encr")

In [ ]:
# Fill in missing overlap as zero overlap
data = data.with_columns(pl.col(["overlap_survey_1","overlap_survey_2","overlap_survey_3","overlap_survey_4"]).fill_null(strategy="zero"))

In [ ]:
# Keep only complete cases and relevant vars
data = data.drop_nulls().drop(["nomem_encr","id_hhd","is_ego_survey"])

In [ ]:
# Calculate cumulative network size
data = data.with_columns(
    net_size_hhd_cum_2 = pl.sum_horizontal(["net_size_hhd_1", "net_size_hhd_2"]),
    net_size_hhd_cum_3 = pl.sum_horizontal(["net_size_hhd_1", "net_size_hhd_2", "net_size_hhd_3"]),
    net_size_hhd_cum_4 = pl.sum_horizontal(["net_size_hhd_1", "net_size_hhd_2", "net_size_hhd_3", "net_size_hhd_4"]),
)

# 2 - Figure 6

In [ ]:
# Prepare data
fig_filter_d1 = 10
fig_filter_d2 = 50
fig_filter_d3 = 50
fig_filter_d4 = 100

data_fig = (
    data
    # .filter(
    #     (pl.col("net_size_hhd_1")<pl.col("net_size_hhd_1").quantile(quantile=0.995, interpolation="nearest")) & 
    #     (pl.col("net_size_hhd_2")<pl.col("net_size_hhd_2").quantile(quantile=0.995, interpolation="nearest")) & 
    #     (pl.col("net_size_hhd_3")<pl.col("net_size_hhd_3").quantile(quantile=0.995, interpolation="nearest")) & 
    #     (pl.col("net_size_hhd_4")<pl.col("net_size_hhd_4").quantile(quantile=0.995, interpolation="nearest"))
    # )
    .filter(
        (pl.col("net_size_hhd_1")<fig_filter_d1) & 
        (pl.col("net_size_hhd_2")<fig_filter_d2) & 
        (pl.col("net_size_hhd_3")<fig_filter_d3) & 
        (pl.col("net_size_hhd_4")<fig_filter_d4)
    )
    .drop(["RINPERSOON","hhd_size","net_size_hhd_2","net_size_hhd_3","net_size_hhd_4"])
)

cols_fig_y = ["y_perc_fam_comp","y_perc_contact_comp","y_perc_supp_comp"]
fig_predictors = data_fig.drop(cols_fig_y).select((pl.all()-pl.all().mean()) / pl.all().std()) # Normalized predictors
fig_outcomes = data_fig.select(cols_fig_y)

In [ ]:
# Linear regressions for figure

list_cols_fig_x = [
    ["net_size_hhd_1","overlap_survey_1"],
    ["net_size_hhd_cum_2","density_2","overlap_survey_2"],
    ["net_size_hhd_cum_3","density_3","overlap_survey_3"],
    ["net_size_hhd_cum_4","density_4","overlap_survey_4"]
]

dict_results = {}

for x in list_cols_fig_x:
    # Identify network distance of predictors
    dist = x[0][-1]
    # Set up regression
    reg_x = fig_predictors[x].to_numpy()
    reg_x = sm.add_constant(reg_x)
    # Create nested dictionary for distance
    dict_results[dist] = {}
    # Iterate over outcomes to fit the model
    for y in cols_fig_y:
        # Create nested dictionary for outcome
        dict_results[dist][y] = {}
        # Set up regression for outcome
        reg_y = fig_outcomes[y].to_numpy()
        reg = sm.OLS(reg_y, reg_x)
        # Fit with robust standard errors
        results = reg.fit()
        results = results.get_robustcov_results(cov_type="HC2")
        # Store results in nested dictionary
        dict_results[dist][y]["predictors"] = x
        dict_results[dist][y]["coefs"] = list(results.params)[1:]
        dict_results[dist][y]["se"] = list(results.bse)[1:]
        dict_results[dist][y]["pvalues"] = list(results.pvalues)[1:]
        dict_results[dist][y]["constant"] = {}
        dict_results[dist][y]["constant"]["coef"] = list(results.params)[0]
        dict_results[dist][y]["constant"]["se"] = list(results.bse)[0]
        dict_results[dist][y]["constant"]["pvalue"] = list(results.pvalues)[0]
        dict_results[dist][y]["rsquared"] = {}
        dict_results[dist][y]["rsquared"]["standard"]= results.rsquared
        dict_results[dist][y]["rsquared"]["adjusted"]= results.rsquared_adj
        dict_results[dist][y]["fstat"] = {} 
        dict_results[dist][y]["fstat"]["stat"] = results.fvalue
        dict_results[dist][y]["fstat"]["pvalue"] = results.f_pvalue
        dict_results[dist][y]["observations"]= results.nobs

In [ ]:
# Store results
with open(config["output"]["dict_regressions"], "w") as file:
    json.dump(dict_results, file, indent=2)

In [ ]:
# Function to create results dataframes for plot
def create_results_df(cols_fig_y, dict_results, dist, type):
    df = pl.DataFrame(
        {
            "predictors":dict_results[str(dist)][cols_fig_y[0]]["predictors"],
            cols_fig_y[0]:dict_results[str(dist)][cols_fig_y[0]][type], 
            cols_fig_y[1]:dict_results[str(dist)][cols_fig_y[1]][type], 
            cols_fig_y[2]:dict_results[str(dist)][cols_fig_y[2]][type]
        }
    )
    # Add a density row to the distance 1 results for plot
    if f"density_{dist}" not in df["predictors"]:
        df_density = pl.DataFrame({"predictors":[f"density_{dist}"],cols_fig_y[0]:[None],cols_fig_y[1]:[None],cols_fig_y[2]:[None]})
        df_size = df.head(1)
        df_overlap = df.tail(1)
        df = pl.concat([df_size,df_density,df_overlap])

    return df

In [ ]:
def flag_significance(df_pvalues):
    '''
    Function to recode numerical pvalues to
    asterisks representing statistical significance.
    '''
    for col in df_pvalues.columns:
        df_pvalues = (
            df_pvalues
            .with_columns(pl.when(pl.col(col)<=0.01).then(100).otherwise(pl.col(col)).alias(col))
            .with_columns(pl.when((pl.col(col)>0.01) & (pl.col(col)<=0.05)).then(200).otherwise(pl.col(col)).alias(col))
            .with_columns(pl.when((pl.col(col)>0.05) & (pl.col(col)<=0.1)).then(300).otherwise(pl.col(col)).alias(col))
            .with_columns(pl.when((pl.col(col)>0.1) & (pl.col(col)<100)).then(400).otherwise(pl.col(col)).alias(col))
            .with_columns(pl.col(col).cast(pl.Int64).cast(pl.String))
            .with_columns(pl.when(pl.col(col)=="100").then(pl.lit("***")).otherwise(pl.col(col)).alias(col))
            .with_columns(pl.when(pl.col(col)=="200").then(pl.lit("**")).otherwise(pl.col(col)).alias(col))
            .with_columns(pl.when(pl.col(col)=="300").then(pl.lit("*")).otherwise(pl.col(col)).alias(col))
            .with_columns(pl.when(pl.col(col)=="400").then(pl.lit("")).otherwise(pl.col(col)).alias(col))
        )
    return df_pvalues

In [ ]:
# Plot
fig_6,ax = plt.subplots(2,2,figsize=(10,3.5), layout="tight")

xticklabels = ["% family", "% contact", "% support"]
yticklabels = ["Size","Density","Overlap"]

cbar_ax = fig_6.add_axes([1,0.21,0.03,0.45])

ax_row = 0
ax_col = 0
for dist in range(1,5):
    # Get coefficients, pvalues, and heatmap annotations
    coefs = create_results_df(cols_fig_y, dict_results, dist, "coefs")
    coefs = coefs.drop("predictors")
    annot = coefs.select(pl.all().round(2).cast(pl.String))
    pvalues = create_results_df(cols_fig_y, dict_results, dist, "pvalues")
    pvalues_stars = flag_significance(pvalues.drop("predictors"))
    annot = annot + pvalues_stars
    # Plot heatmap
    sns.heatmap(
        coefs,
        annot=annot,
        fmt="", # necessary for str to be plottable as annot
        cbar=dist == 4,
        cbar_ax=cbar_ax,
        cmap="coolwarm",
        vmax=5.08,
        vmin=-5.08,
        linewidths = 0.8, 
        linecolor = "white",
        xticklabels=xticklabels,
        yticklabels=yticklabels,
        ax=ax[ax_row,ax_col]
    )
    # Format ticks and titles
    ax[ax_row,ax_col].xaxis.tick_top()
    ax[ax_row,ax_col].tick_params(axis="y",labelrotation=0)
    ax[ax_row,ax_col].set_title(f"Cumulative distance {dist}", fontdict={"fontsize":10})

    # Update ax indexing
    if ax_row==0 and ax_col==0:
        ax_col+=1
    elif ax_row==0 and ax_col==1:
        ax_col = ax_col - 1
        ax_row = ax_row + 1
    elif ax_row==1 and ax_col==0:
        ax_col+=1

In [ ]:
# Save figure
fig_6_out = fig_6.get_figure()
fig_6_out.savefig(config["output"]["fig_6_regression"], bbox_inches = "tight", dpi = 400)